In [1]:
import json
from collections import Counter

In [2]:
import json
from collections import Counter

# Specify the file path
file_path = 'isotretinoin.json'

# Open the file and load its content
with open(file_path, 'r') as file:
    data = json.load(file)

# # Now, 'data' contains the content of the JSON file
# print(len(data))

# # Extract all reactions
# all_reactions = [reaction['reactionmeddrapt'] for record in data for reaction in record['reactions']]

# # Count occurrences of each reaction
# reaction_counts = Counter(all_reactions)

# # Sort reactions by count in descending order
# sorted_reactions = sorted(reaction_counts.items(), key=lambda item: item[1], reverse=True)

# # Convert to dictionary and print
# sorted_reaction_dict = dict(sorted_reactions)

In [12]:
# Initialize dictionaries for reaction counts
reaction_counts_under_18 = {}
reaction_counts_over_18 = {}

for entry in data:
    age = entry.get('patientonsetage', None)
    age_unit = entry.get('patientonsetageunit', None)
    patientsex = entry.get('patientsex', None)
    reactions = entry['reactions']
    
    # We only care about age unit 801 (corresponding to years)
    if age is not None and age_unit == "801":
        if int(age) <= 18:
            for reaction in reactions:
                reaction_term = reaction['reactionmeddrapt']
                reaction_counts_under_18[reaction_term] = reaction_counts_under_18.get(reaction_term, 0) + 1
        else:
            for reaction in reactions:
                reaction_term = reaction['reactionmeddrapt']
                reaction_counts_over_18[reaction_term] = reaction_counts_over_18.get(reaction_term, 0) + 1

# Sort reaction counts in descending order
sorted_reaction_counts_under_18 = dict(sorted(reaction_counts_under_18.items(), key=lambda item: item[1], reverse=True))
sorted_reaction_counts_over_18 = dict(sorted(reaction_counts_over_18.items(), key=lambda item: item[1], reverse=True))

# Display the sorted reaction counts
print("Reactions for patients 18 and under:")
for reaction, count in sorted_reaction_counts_under_18.items():
    print(f"{reaction}: {count}")

print("\nReactions for patients over 18:")
for reaction, count in sorted_reaction_counts_over_18.items():
    print(f"{reaction}: {count}")

Reactions for patients 18 and under:
Off label use: 674
Depression: 591
Product dose omission issue: 461
Suicidal ideation: 292
Arthralgia: 292
Mood altered: 278
Headache: 268
Dry skin: 251
Treatment noncompliance: 206
Product administration interrupted: 205
Anxiety: 191
Lip dry: 171
Adverse drug reaction: 160
Blood triglycerides increased: 154
Epistaxis: 152
Exposure during pregnancy: 151
Back pain: 146
Depressed mood: 146
Drug ineffective: 136
Acne: 132
Mood swings: 127
Myalgia: 125
Nausea: 121
Fatigue: 113
Inflammatory bowel disease: 111
Colitis ulcerative: 98
Suicide attempt: 92
Rash: 85
Unintended pregnancy: 84
Alanine aminotransferase increased: 81
Aspartate aminotransferase increased: 80
Crying: 79
Therapy cessation: 76
Anger: 76
Chapped lips: 71
Irritability: 70
Vomiting: 70
Pyrexia: 67
Pain: 66
Blood cholesterol increased: 63
Epiphyses premature fusion: 63
Aggression: 62
Dizziness: 62
Abdominal pain: 60
Cheilitis: 58
Feeling abnormal: 58
Vision blurred: 56
Insomnia: 55
Crohn^s

In [15]:

from scipy.stats import ttest_ind
import numpy as np

# Let's limit our analysis to the top 20 reactions in the 18 and under category
top_20_reactions_under_18 = list(sorted_reaction_counts_under_18.keys())[:20]

# Initialize arrays to store proportions
proportions_under_18 = []
proportions_over_18 = []

# Calculate total counts for normalization
total_under_18 = sum(reaction_counts_under_18.values())
total_over_18 = sum(reaction_counts_over_18.values())

# Collect proportions for the top 20 reactions and prepare them for T-test
for reaction in top_20_reactions_under_18:
    count_under_18 = reaction_counts_under_18.get(reaction, 0)
    count_over_18 = reaction_counts_over_18.get(reaction, 0)
    
    proportion_under_18 = count_under_18 / total_under_18
    proportion_over_18 = count_over_18 / total_over_18
    
    proportions_under_18.append(proportion_under_18)
    proportions_over_18.append(proportion_over_18)
    
    print(f"{reaction}: Under 18 - {proportion_under_18:.4f}, Over 18 - {proportion_over_18:.4f}")

# Convert the lists to numpy arrays for the T-test
proportions_under_18 = np.array(proportions_under_18)
proportions_over_18 = np.array(proportions_over_18)

# Perform T-test for each reaction
for i, reaction in enumerate(top_20_reactions_under_18):
    t_stat, p_value = ttest_ind([proportions_under_18[i]], [proportions_over_18[i]], equal_var=False)
    print(f"T-test for {reaction}: t-stat={t_stat[0]:.4f}, p-value={p_value[0]:.4f}")




Off label use: Under 18 - 0.0489, Over 18 - 0.0419
Depression: Under 18 - 0.0429, Over 18 - 0.0251
Product dose omission issue: Under 18 - 0.0334, Over 18 - 0.0236
Suicidal ideation: Under 18 - 0.0212, Over 18 - 0.0113
Arthralgia: Under 18 - 0.0212, Over 18 - 0.0171
Mood altered: Under 18 - 0.0202, Over 18 - 0.0093
Headache: Under 18 - 0.0194, Over 18 - 0.0226
Dry skin: Under 18 - 0.0182, Over 18 - 0.0219
Treatment noncompliance: Under 18 - 0.0149, Over 18 - 0.0170
Product administration interrupted: Under 18 - 0.0149, Over 18 - 0.0106
Anxiety: Under 18 - 0.0139, Over 18 - 0.0120
Lip dry: Under 18 - 0.0124, Over 18 - 0.0110
Adverse drug reaction: Under 18 - 0.0116, Over 18 - 0.0098
Blood triglycerides increased: Under 18 - 0.0112, Over 18 - 0.0102
Epistaxis: Under 18 - 0.0110, Over 18 - 0.0069
Exposure during pregnancy: Under 18 - 0.0110, Over 18 - 0.0464
Back pain: Under 18 - 0.0106, Over 18 - 0.0071
Depressed mood: Under 18 - 0.0106, Over 18 - 0.0046
Drug ineffective: Under 18 - 0.00

/var/folders/hz/dpjg24vs7w3b1dlswwlm5l3c0000gn/T/ipykernel_32399/2075465583.py:34: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = ttest_ind([proportions_under_18[i]], [proportions_over_18[i]], equal_var=False)
/usr/local/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


IndexError: invalid index to scalar variable.

In [8]:
import scipy.stats as stats

# Calculate the total number of reactions for under 18 and over 18
total_reactions_under_18 = sum(reaction_counts_under_18.values())
total_reactions_over_18 = sum(reaction_counts_over_18.values())

# Get the top 20 reactions for under 18
top_20_reactions_under_18 = list(sorted_reaction_counts_under_18.keys())
t_test_results = {}

# Perform T tests for each of the top 20 reactions
for reaction in top_20_reactions_under_18:
    count_under_18 = reaction_counts_under_18.get(reaction, 0)
    prop_under_18 = count_under_18 / total_reactions_under_18
    
    count_over_18 = reaction_counts_over_18.get(reaction, 0)
    prop_over_18 = count_over_18 / total_reactions_over_18
    
    # Calculate the observed counts for T test
    observed = [
        [count_under_18, total_reactions_under_18 - count_under_18],
        [count_over_18, total_reactions_over_18 - count_over_18]
    ]
    
    # Perform the T test
    chi2_stat, p_val, dof, ex = stats.chi2_contingency(observed, correction=False)
    
    # Store the results
    t_test_results[reaction] = {'p_value': p_val, 'sample_size_under_18': total_reactions_under_18, 'sample_size_over_18': total_reactions_over_18}

# Display the proportion of each reaction, the p-value from T test, and the total sample size
print("Reaction proportions, T test results, and Total Sample Size:")
for reaction in top_20_reactions_under_18:
    prop_under_18 = reaction_counts_under_18.get(reaction, 0) / total_reactions_under_18
    prop_over_18 = reaction_counts_over_18.get(reaction, 0) / total_reactions_over_18
    result = t_test_results.get(reaction, {'p_value': ">0.05", 'sample_size_under_18': total_reactions_under_18, 'sample_size_over_18': total_reactions_over_18})
    
    print(f"{reaction}: under 18 - {prop_under_18:.2%}, over 18 - {prop_over_18:.2%}, p-value: {result['p_value']:.4f}, "
          f"Sample Size (Under 18): {result['sample_size_under_18']}, Sample Size (Over 18): {result['sample_size_over_18']}")


Reaction proportions, T test results, and Total Sample Size:
Off label use: under 18 - 4.89%, over 18 - 4.19%, p-value: 0.0023, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Depression: under 18 - 4.29%, over 18 - 2.51%, p-value: 0.0000, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Product dose omission issue: under 18 - 3.34%, over 18 - 2.36%, p-value: 0.0000, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Suicidal ideation: under 18 - 2.12%, over 18 - 1.13%, p-value: 0.0000, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Arthralgia: under 18 - 2.12%, over 18 - 1.71%, p-value: 0.0073, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Mood altered: under 18 - 2.02%, over 18 - 0.93%, p-value: 0.0000, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Headache: under 18 - 1.94%, over 18 - 2.26%, p-value: 0.0490, Sample Size (Under 18): 13782, Sample Size (Over 18): 19552
Dry skin: under 18 - 1.82%, over 18 - 2.19%,

# Split by sex

In [11]:
# Initialize dictionaries for reaction counts
reaction_counts_under_18_male = {}
reaction_counts_under_18_female = {}
reaction_counts_over_18_male = {}
reaction_counts_over_18_female = {}

for entry in data:
    age = entry.get('patientonsetage', None)
    age_unit = entry.get('patientonsetageunit', None)
    patientsex = entry.get('patientsex', None)
    reactions = entry.get('reactions', [])

    # Exclude entries that don't conform
    if age is not None and age_unit == "801" and patientsex in ["1", "2"]:
        age = int(age)
        if age <= 18:
            if patientsex == "1":
                for reaction in reactions:
                    reaction_term = reaction.get('reactionmeddrapt', None)
                    if reaction_term:
                        reaction_counts_under_18_male[reaction_term] = reaction_counts_under_18_male.get(reaction_term, 0) + 1
            elif patientsex == "2":
                for reaction in reactions:
                    reaction_term = reaction.get('reactionmeddrapt', None)
                    if reaction_term:
                        reaction_counts_under_18_female[reaction_term] = reaction_counts_under_18_female.get(reaction_term, 0) + 1
        else:
            if patientsex == "1":
                for reaction in reactions:
                    reaction_term = reaction.get('reactionmeddrapt', None)
                    if reaction_term:
                        reaction_counts_over_18_male[reaction_term] = reaction_counts_over_18_male.get(reaction_term, 0) + 1
            elif patientsex == "2":
                for reaction in reactions:
                    reaction_term = reaction.get('reactionmeddrapt', None)
                    if reaction_term:
                        reaction_counts_over_18_female[reaction_term] = reaction_counts_over_18_female.get(reaction_term, 0) + 1

# Sort reaction counts in descending order
sorted_reaction_counts_under_18_male = dict(sorted(reaction_counts_under_18_male.items(), key=lambda item: item[1], reverse=True))
sorted_reaction_counts_under_18_female = dict(sorted(reaction_counts_under_18_female.items(), key=lambda item: item[1], reverse=True))
sorted_reaction_counts_over_18_male = dict(sorted(reaction_counts_over_18_male.items(), key=lambda item: item[1], reverse=True))
sorted_reaction_counts_over_18_female = dict(sorted(reaction_counts_over_18_female.items(), key=lambda item: item[1], reverse=True))

# Display the sorted reaction counts
print("Reactions for patients 18 and under - Male:")
for reaction, count in sorted_reaction_counts_under_18_male.items():
    print(f"{reaction}: {count}")

print("\nReactions for patients 18 and under - Female:")
for reaction, count in sorted_reaction_counts_under_18_female.items():
    print(f"{reaction}: {count}")

print("\nReactions for patients over 18 - Male:")
for reaction, count in sorted_reaction_counts_over_18_male.items():
    print(f"{reaction}: {count}")

print("\nReactions for patients over 18 - Female:")
for reaction, count in sorted_reaction_counts_over_18_female.items():
    print(f"{reaction}: {count}")


Reactions for patients 18 and under - Male:
Off label use: 334
Depression: 305
Suicidal ideation: 180
Arthralgia: 175
Mood altered: 139
Dry skin: 137
Lip dry: 108
Headache: 106
Anxiety: 96
Epistaxis: 93
Inflammatory bowel disease: 85
Myalgia: 80
Acne: 78
Depressed mood: 76
Blood triglycerides increased: 73
Fatigue: 72
Colitis ulcerative: 71
Back pain: 69
Product dose omission issue: 68
Drug ineffective: 68
Mood swings: 65
Anger: 57
Aggression: 54
Aspartate aminotransferase increased: 54
Treatment noncompliance: 51
Alanine aminotransferase increased: 50
Irritability: 49
Erectile dysfunction: 49
Adverse drug reaction: 47
Suicide attempt: 45
Nausea: 45
Pyrexia: 44
Epiphyses premature fusion: 43
Rash: 40
Erythema: 39
Chapped lips: 39
Crohn^s disease: 39
Insomnia: 38
Acne fulminans: 38
Pain: 38
Cheilitis: 37
Disturbance in attention: 35
Completed suicide: 34
Feeling abnormal: 33
Crying: 32
Neutrophilic dermatosis: 31
Blood cholesterol increased: 31
Therapy cessation: 30
Abdominal pain: 30
D

In [15]:
import scipy.stats as stats

# Calculate the total number of reactions for under 18 and over 18
total_reactions_under_18 = sum(reaction_counts_under_18_female.values())
total_reactions_over_18 = sum(reaction_counts_over_18_female.values())

# Get the top 20 reactions for under 18
top_20_reactions_under_18 = list(sorted_reaction_counts_under_18_female.keys())
t_test_results = {}

# Perform T tests for each of the top 20 reactions
for reaction in top_20_reactions_under_18:
    count_under_18 = reaction_counts_under_18.get(reaction, 0)
    prop_under_18 = count_under_18 / total_reactions_under_18
    
    count_over_18 = reaction_counts_over_18.get(reaction, 0)
    prop_over_18 = count_over_18 / total_reactions_over_18
    
    # Calculate the observed counts for T test
    observed = [
        [count_under_18, total_reactions_under_18 - count_under_18],
        [count_over_18, total_reactions_over_18 - count_over_18]
    ]
    
    # Perform the T test
    chi2_stat, p_val, dof, ex = stats.chi2_contingency(observed, correction=False)
    
    # Store the results
    t_test_results[reaction] = {'p_value': p_val, 'sample_size_under_18': total_reactions_under_18, 'sample_size_over_18': total_reactions_over_18}

# Display the proportion of each reaction, the p-value from T test, and the total sample size
print("FEMALE Reaction proportions, T test results, and Total Sample Size:")
for reaction in top_20_reactions_under_18:
    prop_under_18 = reaction_counts_under_18.get(reaction, 0) / total_reactions_under_18
    prop_over_18 = reaction_counts_over_18.get(reaction, 0) / total_reactions_over_18
    result = t_test_results.get(reaction, {'p_value': ">0.05", 'sample_size_under_18': total_reactions_under_18, 'sample_size_over_18': total_reactions_over_18})
    
    print(f"{reaction}: under 18 - {prop_under_18:.2%}, over 18 - {prop_over_18:.2%}, p-value: {result['p_value']:.4f}, "
          f"Sample Size (Under 18): {result['sample_size_under_18']}, Sample Size (Over 18): {result['sample_size_over_18']}")


FEMALE Reaction proportions, T test results, and Total Sample Size:
Off label use: under 18 - 11.16%, over 18 - 5.79%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Product dose omission issue: under 18 - 7.63%, over 18 - 3.26%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Depression: under 18 - 9.78%, over 18 - 3.46%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Product administration interrupted: under 18 - 3.39%, over 18 - 1.46%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Exposure during pregnancy: under 18 - 2.50%, over 18 - 6.42%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Headache: under 18 - 4.44%, over 18 - 3.12%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Mood altered: under 18 - 4.60%, over 18 - 1.29%, p-value: 0.0000, Sample Size (Under 18): 6041, Sample Size (Over 18): 14148
Treatment n